# Perform regression on parameterized surfaces

## Imports

In [1]:
# imports

import trimesh
import os
import sys
import subprocess
import torch
import numpy as np
os.environ["GEOMSTATS_BACKEND"] = "pytorch"
import geomstats.backend as gs
import my28brains.discrete_surfaces

INFO: Using pytorch backend


In [2]:
gitroot_path = subprocess.check_output(
    ["git", "rev-parse", "--show-toplevel"], universal_newlines=True
)
os.chdir(gitroot_path[:-1])
work_dir = os.getcwd()

my28brains_dir = os.path.join(work_dir, "my28brains")

sys_dir = os.path.dirname(work_dir)
sys.path.append(sys_dir)
sys.path.append(my28brains_dir)

# this forces the notebook to re-load
import importlib
importlib.reload(my28brains.discrete_surfaces)

from my28brains.discrete_surfaces import (
    DiscreteSurfaces,
    ElasticMetric,
)

import my28brains.default_config as default_config

## Generate parameterized geodesic from sphere to ellipse using `discrete_curves.py`.

In [3]:
#create sphere mesh
sphere = trimesh.creation.icosphere(subdivisions=3, radius=30.0)

#Create ellipsoid mesh
# Create a scaling matrix for the semi-axes lengths
scales = np.array([2, 2, 3])
scale_matrix = np.diag(scales)

scale_matrix = gs.array(scale_matrix)

# Apply the scaling transformation to the mesh vertices
scaled_vertices = sphere.vertices.dot(scale_matrix)

# Create a new mesh with the scaled vertices
ellipsoid = trimesh.Trimesh(vertices=scaled_vertices, faces=sphere.faces)

#Define Surface Space
SURFACE_SPACE = DiscreteSurfaces(faces = sphere.faces)

METRIC = ElasticMetric(
    space = SURFACE_SPACE,
    a0=default_config.a0,
    a1=default_config.a1,
    b1=default_config.b1,
    c1=default_config.c1,
    d1=default_config.d1,
    a2=default_config.a2,)

# CREATE GEODESIC
n_vertices = sphere.vertices.shape[0]
print(n_vertices)


def geodesics_sphere_to_ellipsoid(
    n_geodesics=1, n_times=5
):
    """Generate a dataset of geodesics that transform spheres into ellipsoids."""
    dim = 3
    
    geodesics = gs.zeros((n_geodesics, n_times, n_vertices, dim))
    times = gs.arange(0, 1, 1 / n_times)
    for i_geodesic in range(n_geodesics):
        start_surface = gs.array(sphere.vertices)
        end_surface = gs.array(ellipsoid.vertices)
        print(start_surface.shape)
        geodesic = METRIC.geodesic(initial_point = start_surface, end_point = end_surface)
        
        geodesics[i_geodesic] = geodesic(times)

    return geodesics, times

initial_surface = gs.array(sphere.vertices)
end_surface = gs.array(ellipsoid.vertices)
geodesics, times = geodesics_sphere_to_ellipsoid()
geodesic = geodesics[0]
true_intercept = initial_surface
true_coef = METRIC.log(end_surface, initial_surface)
print("True intercept: "+str(true_intercept))
print("True coef: "+str(true_coef))

642
torch.Size([642, 3])
before dimension expansion:
midpoints.shape torch.Size([3, 642, 3])
initial_point.shape torch.Size([642, 3])
end_point.shape torch.Size([642, 3])
after dimension expansion:
midpoints.shape torch.Size([1, 3, 642, 3])
initial_point.shape torch.Size([1, 642, 3])
end_point.shape torch.Size([1, 642, 3])
num_points 1
one_midpoint torch.Size([3, 642, 3])
one_initial_point torch.Size([642, 3])
one_endpoint torch.Size([642, 3])
INNER PRODUCT
h torch.Size([1, 642, 3])
base_point torch.Size([1, 642, 3])
INNER PRODUCT
h torch.Size([1, 642, 3])
base_point torch.Size([1, 642, 3])
INNER PRODUCT
h torch.Size([1, 642, 3])
base_point torch.Size([1, 642, 3])
INNER PRODUCT
h torch.Size([1, 642, 3])
base_point torch.Size([1, 642, 3])
RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =         5778     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  3.86728D+08    |proj g|=  4.99244D+04
one_midpoint torch.Size

 This problem is unconstrained.


one_midpoint torch.Size([3, 642, 3])
one_initial_point torch.Size([642, 3])
one_endpoint torch.Size([642, 3])
INNER PRODUCT
h torch.Size([1, 642, 3])
base_point torch.Size([1, 642, 3])
INNER PRODUCT
h torch.Size([1, 642, 3])
base_point torch.Size([1, 642, 3])
INNER PRODUCT
h torch.Size([1, 642, 3])
base_point torch.Size([1, 642, 3])
INNER PRODUCT
h torch.Size([1, 642, 3])
base_point torch.Size([1, 642, 3])

At iterate    1    f=  3.15951D+08    |proj g|=  1.37249D+05

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
 5778      1      5      1     0     0   1.372D+05   3.160D+08
  F =   315951016.62582248     

CONVERGENCE:

 This problem is unconstrained.


# GEODESIC REGRESSION

In [4]:
# Regression imports

import matplotlib.pyplot as plt

import geomstats.backend as gs
import geomstats.visualization as visualization
from geomstats.geometry.special_euclidean import SpecialEuclidean
from geomstats.learning.frechet_mean import FrechetMean, variance
from geomstats.learning.geodesic_regression import GeodesicRegression

In [ ]:
n_times = len(times)


# maxiter was 100
gr = GeodesicRegression(
        SURFACE_SPACE,
        metric=METRIC,
        center_X=False,
        method="riemannian",
        max_iter=5,
        init_step_size=0.1,
        verbose=True,
        initialization="frechet",
    )

# X = gs.random.normal(size=(5))
# X -= gs.mean(X)
# print(geodesic.shape)

gr.fit(times, geodesic, compute_training_score=False)

intercept_hat, coef_hat = gr.intercept_, gr.coef_

LOG before squeeze:
point.shape:  torch.Size([5, 642, 3])
base_point.shape:  torch.Size([642, 3])
one_point.shape:  torch.Size([642, 3])
one_base_point.shape:  torch.Size([642, 3])
before dimension expansion:
midpoints.shape torch.Size([3, 642, 3])
initial_point.shape torch.Size([642, 3])
end_point.shape torch.Size([642, 3])
after dimension expansion:
midpoints.shape torch.Size([1, 3, 642, 3])
initial_point.shape torch.Size([1, 642, 3])
end_point.shape torch.Size([1, 642, 3])
num_points 1
one_midpoint torch.Size([3, 642, 3])
one_initial_point torch.Size([642, 3])
one_endpoint torch.Size([642, 3])
INNER PRODUCT
h torch.Size([1, 642, 3])
base_point torch.Size([1, 642, 3])
INNER PRODUCT
h torch.Size([1, 642, 3])
base_point torch.Size([1, 642, 3])
INNER PRODUCT
h torch.Size([1, 642, 3])
base_point torch.Size([1, 642, 3])
INNER PRODUCT
h torch.Size([1, 642, 3])
base_point torch.Size([1, 642, 3])
RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =         5778    

 This problem is unconstrained.
 This problem is unconstrained.


one_midpoint torch.Size([3, 642, 3])
one_initial_point torch.Size([642, 3])
one_endpoint torch.Size([642, 3])
INNER PRODUCT
h torch.Size([1, 642, 3])
base_point torch.Size([1, 642, 3])
INNER PRODUCT
h torch.Size([1, 642, 3])
base_point torch.Size([1, 642, 3])
INNER PRODUCT
h torch.Size([1, 642, 3])
base_point torch.Size([1, 642, 3])
INNER PRODUCT
h torch.Size([1, 642, 3])
base_point torch.Size([1, 642, 3])

At iterate    1    f=  1.63282D+05    |proj g|=  3.39690D+00

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
 5778      1      3      1     0     0   3.397D+00   1.633D+05
  F =   163281.84661218108     

CONVERGENCE:

 This problem is unconstrained.


one_midpoint torch.Size([3, 642, 3])
one_initial_point torch.Size([642, 3])
one_endpoint torch.Size([642, 3])
INNER PRODUCT
h torch.Size([1, 642, 3])
base_point torch.Size([1, 642, 3])
INNER PRODUCT
h torch.Size([1, 642, 3])
base_point torch.Size([1, 642, 3])
INNER PRODUCT
h torch.Size([1, 642, 3])
base_point torch.Size([1, 642, 3])
INNER PRODUCT
h torch.Size([1, 642, 3])
base_point torch.Size([1, 642, 3])

At iterate    1    f=  6.64867D+05    |proj g|=  1.47768D+01

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
 5778      1      3      1     0     0   1.478D+01   6.649D+05
  F =   664867.38449961739     

CONVERGENCE:

...

INNER PRODUCT
h torch.Size([1, 642, 3])
base_point torch.Size([1, 642, 3])
INNER PRODUCT
h torch.Size([1, 642, 3])
base_point torch.Size([1, 642, 3])
INNER PRODUCT
h torch.Size([1, 642, 3])
base_point torch.Size([1, 642, 3])

At iterate    1    f=  1.93614D-01    |proj g|=  5.93244D+02

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
 1926      1      3      1     0     0   5.932D+02   1.936D-01
  F =  0.19361378577381289     

CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH             
INNER PRODUCT
h torch.Size([1, 642, 3])
base_point torch.Size([1, 642, 3])
INNER PRODUCT
h torch.Size([1, 642, 3])
base_point torch.Size

 This problem is unconstrained.
 This problem is unconstrained.


INNER PRODUCT
h torch.Size([1, 642, 3])
base_point torch.Size([1, 642, 3])
INNER PRODUCT
h torch.Size([1, 642, 3])
base_point torch.Size([1, 642, 3])

At iterate    1    f=  1.97120D-01    |proj g|=  5.97099D+02

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
 1926      1      3      1     0     0   5.971D+02   1.971D-01
  F =  0.19711953766949103     

CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH             
INNER PRODUCT
h torch.Size([1, 642, 3])
base_point torch.Size([1, 642, 3])
INNER PRODUCT
h torch.Size([1, 642, 3])
base_point torch.Size([1, 642, 3])
INNER PRODUCT
h torch.Size([1, 642, 3])
base_point torch.Size

 This problem is unconstrained.


one_midpoint torch.Size([3, 642, 3])
one_initial_point torch.Size([642, 3])
one_endpoint torch.Size([642, 3])
INNER PRODUCT
h torch.Size([1, 642, 3])
base_point torch.Size([1, 642, 3])
INNER PRODUCT
h torch.Size([1, 642, 3])
base_point torch.Size([1, 642, 3])
INNER PRODUCT
h torch.Size([1, 642, 3])
base_point torch.Size([1, 642, 3])
INNER PRODUCT
h torch.Size([1, 642, 3])
base_point torch.Size([1, 642, 3])

At iterate    1    f=  3.44214D+06    |proj g|=  8.50585D+01

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
 5778      1      3      1     0     0   8.506D+01   3.442D+06
  F =   3442142.4751160685     

CONVERGENCE:

In [ ]:
true_intercept_mesh = trimesh.Trimesh(vertices= true_intercept,
                       faces=sphere.faces)

intercept_hat_mesh = trimesh.Trimesh(vertices= intercept_hat,
                                     faces=sphere.faces)

In [ ]:
true_intercept_mesh.show()

In [ ]:
intercept_hat_mesh.show()
